<a href="https://colab.research.google.com/github/zumaku/tensorflow_journey/blob/main/chess_detection_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Langkah 1: Persiapan Lingkungan di Google Colab
Pertama, kita akan memulai dengan mengimpor pustaka yang dibutuhkan dan menghubungkan Google Colab dengan Google Drive.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
from google.colab import drive

In [ ]:
# Menghubungkan Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Menentukan direktori dataset
dataset_dir = '/content/drive/MyDrive/Tensorflow Journey/Chessman-image-dataset'

# Langkah 2: Mempersiapkan Dataset
Setelah itu, kita akan mempersiapkan dataset untuk digunakan dalam pelatihan model.

**Alur Kerja**
1. Normalisasi:
  
  Gambar yang dimuat akan dinormalisasi dengan skala 1.0/255.0 sehingga nilai piksel berada dalam rentang 0-1.

2. Pembagian Dataset:

  Dataset akan secara otomatis dibagi menjadi subset pelatihan dan validasi berdasarkan parameter validation_split.

3. Batch Processing:

  Gambar akan diproses dalam batch (32 gambar per batch) untuk efisiensi memori dan performa pelatihan.

4. Pengubahan Ukuran Gambar:

  Setiap gambar akan diubah ukurannya menjadi 150x150 piksel sebelum dimasukkan ke dalam model.

5. One-Hot Encoding:

  Label kelas gambar akan diubah menjadi one-hot encoded vectors untuk digunakan dalam klasifikasi multi-kelas.

In [ ]:
# Mengatur parameter ImageDataGenerator
datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    validation_split=0.2  # 20% data digunakan untuk validasi
)

`ImageDataGenerator` adalah kelas di Keras yang digunakan untuk mempersiapkan dan mengaugmentasi gambar sebelum diberikan ke model.

1. Parameter `rescale=1.0/255.0`:
  
  Mengubah skala nilai piksel gambar dari 0-255 menjadi 0-1 dengan membagi setiap nilai piksel dengan 255. Ini adalah langkah normalisasi yang umum dalam pengolahan citra.

2. Parameter `validation_split=0.2`:

  Menentukan bahwa 20% dari data akan digunakan untuk validasi, sedangkan 80% sisanya untuk pelatihan.

In [ ]:
# Memuat dataset untuk pelatihan dan validasi
train_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # Menggunakan subset pelatihan
)

validation_generator = datagen.flow_from_directory(
    dataset_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # Menggunakan subset validasi
)

Found 442 images belonging to 6 classes.
Found 109 images belonging to 6 classes.


Fungsi `flow_from_directory` adalah metode dari ImageDataGenerator yang digunakan untuk memuat data gambar dari direktori yang terstruktur dengan baik. Data harus disusun dalam subdirektori, di mana nama subdirektori tersebut akan menjadi label kelas.

1. Parameter `dataset_dir`:
    
  Path ke direktori utama dataset yang diatur dalam folder terpisah berdasarkan label buah catur.

2. Parameter `target_size=(150, 150)`:

  Ukuran target untuk mengubah ukuran setiap gambar menjadi 150x150 piksel. Ini memastikan bahwa semua gambar memiliki ukuran yang konsisten saat dimasukkan ke dalam model.

3. Parameter `batch_size=32`:

  Jumlah gambar yang akan diproses sekaligus dalam satu batch. Batch size mempengaruhi penggunaan memori dan performa pelatihan.

4. Parameter `class_mode='categorical'`:

  Menentukan bahwa label kelas akan diubah menjadi one-hot encoded vectors yang digunakan untuk klasifikasi multi-kelas.

5. Parameter `subset='training'` dan `subset='validation'`:

  Menentukan apakah subset data ini digunakan untuk pelatihan atau validasi berdasarkan validation_split yang telah diatur sebelumnya.

Variable `train_generator`: Generator yang menghasilkan batch data pelatihan dari direktori. Menggunakan 80% dari dataset berdasarkan validation_split.

Variable `validation_generator`: Generator yang menghasilkan batch data validasi dari direktori. Menggunakan 20% dari dataset berdasarkan validation_split.

# Langkah 3: Membangun Model TensorFlow
Selanjutnya, kita akan membangun arsitektur model menggunakan Sequential API dari Keras.

In [ ]:
# Membangun model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

Model `Sequential` adalah jenis model dalam Keras yang memungkinkan Anda untuk membangun model lapis demi lapis secara berurutan.

1. `Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3))`:

  *   `Conv2D`: Lapisan konvolusi yang menerapkan 32 filter (kernel) dengan ukuran 3x3 pada gambar input.
  *   `activation='relu'`: Fungsi aktivasi ReLU (Rectified Linear Unit) yang digunakan untuk memperkenalkan non-linearitas ke dalam model.
  *   `input_shape=(150, 150, 3)`: Bentuk input gambar, di mana 150x150 adalah dimensi gambar, dan 3 adalah jumlah saluran warna (RGB).

2. `MaxPooling2D(2, 2)`:

  *   `MaxPooling2D`: Lapisan pooling yang mengurangi dimensi gambar dengan mengambil nilai maksimum dalam setiap blok 2x2. Ini membantu mengurangi ukuran data dan mengurangi overfitting.

3. `Conv2D(64, (3, 3), activation='relu')`:

  *   Lapisan konvolusi kedua dengan 64 filter dan ukuran kernel 3x3.

4. `MaxPooling2D(2, 2)`:

  *   Lapisan pooling kedua dengan blok 2x2.

5. `Conv2D(128, (3, 3), activation='relu')`:

  *   Lapisan konvolusi ketiga dengan 128 filter dan ukuran kernel 3x3.

6. `MaxPooling2D(2, 2)`:

  *   Lapisan pooling ketiga dengan blok 2x2.

7. `Flatten()`:

  *   `Flatten`: Mengubah output dari lapisan sebelumnya menjadi vektor satu dimensi. Ini diperlukan sebelum memasukkannya ke dalam lapisan Dense.

8. `Dense(512, activation='relu')`:

  *   `Dense`: Lapisan fully connected dengan 512 neuron dan fungsi aktivasi ReLU. Lapisan ini berfungsi untuk menggabungkan fitur-fitur yang telah diekstraksi oleh lapisan konvolusi sebelumnya.
  
9. `Dropout(0.5)`:

  *   `Dropout`: Lapisan dropout dengan probabilitas 50%. Dropout membantu mencegah overfitting dengan mengabaikan (meng-drop) neuron secara acak selama pelatihan.

10. `Dense(train_generator.num_classes, activation='softmax')`:

  *   Lapisan output dengan jumlah neuron sama dengan jumlah kelas dalam dataset (diambil dari `train_generator.num_classes`), menggunakan fungsi aktivasi softmax untuk menghasilkan probabilitas untuk setiap kelas.



In [ ]:
# Mengompilasi model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Langkah 4: Melatih Model
Sekarang kita akan melatih model dengan data yang telah kita siapkan.

In [ ]:
# Melatih model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=20
)

Epoch 1/20
13/13 [==============================] - 251s 19s/step - loss: 1.9702 - accuracy: 0.1488 - val_loss: 1.7740 - val_accuracy: 0.1875
Epoch 2/20
13/13 [==============================] - 36s 3s/step - loss: 1.7235 - accuracy: 0.2732 - val_loss: 1.7478 - val_accuracy: 0.2083
Epoch 3/20
13/13 [==============================] - 33s 2s/step - loss: 1.6165 - accuracy: 0.3585 - val_loss: 1.7818 - val_accuracy: 0.2604
Epoch 4/20
13/13 [==============================] - 34s 3s/step - loss: 1.4273 - accuracy: 0.4268 - val_loss: 1.7135 - val_accuracy: 0.3438
Epoch 5/20
13/13 [==============================] - 34s 3s/step - loss: 1.1836 - accuracy: 0.5585 - val_loss: 1.7543 - val_accuracy: 0.3750
Epoch 6/20
13/13 [==============================] - 35s 3s/step - loss: 0.9744 - accuracy: 0.6561 - val_loss: 1.9388 - val_accuracy: 0.3542
Epoch 7/20
13/13 [==============================] - 33s 3s/step - loss: 0.8869 - accuracy: 0.6902 - val_loss: 2.0334 - val_accuracy: 0.3333
Epoch 8/20
13/13 [

# Langkah 5: Evaluasi dan Validasi
Kita akan mengevaluasi performa model dan menyimpan model yang telah dilatih.

In [ ]:
# Mengevaluasi model
loss, accuracy = model.evaluate(validation_generator)
print(f'Validation loss: {loss}')
print(f'Validation accuracy: {accuracy}')

4/4 [==============================] - 4s 642ms/step - loss: 3.8270 - accuracy: 0.4220
Validation loss: 3.8269615173339844
Validation accuracy: 0.4220183491706848


In [ ]:
# Menyimpan model
# model.save('/content/drive/My Drive/model_catur.h5')

# Kesimpulan

Berdasarkan hasil evaluasi yang Anda sebutkan, yaitu `Validation loss: 3.8269615173339844` dan `Validation accuracy: 0.4220183491706848`, bisa dikatakan bahwa model tersebut masih kurang baik. Dengan akurasi validasi sekitar 42%, model tersebut masih memiliki ruang untuk perbaikan yang signifikan.

Sedingga itu perlu adanya perbaikan model, baik berupa perbaikan pada augmentasi data atau perbaikan pada arsitektur.